# Server

In [1]:
import requests.exceptions
import tekore as tk
from dotenv import load_dotenv
import httpx

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)

# trans = httpx.HTTPTransport(retries=3)
# client = httpx.Client(timeout=30, transport=trans)
# sender = tk.SyncSender(client=client)
spotify = tk.Spotify()

host = '127.0.0.1'
port = 5000

In [2]:
from os import environ as env
import json
from authentication.spotify_server import SpotifyServer
try:
    with open(".spotify_credentials.json", "r") as infile:
        token_dict = json.load(infile)
        token_dict["scope"] = " ".join(token_dict["scope"])
        spotify.token = tk.Token(token_dict, token_dict["uses_pkce"])
        spotify.token = cred.refresh(spotify.token)
except Exception as e:
    print(e)
    app = SpotifyServer(host, port, spotify, cred)
    spotify.token = app.spawn_single_use_server()

In [3]:
from typing import List
token: tk.Token = spotify.token
scopes: List[str] = list(token.scope)
token_dict = {
    "token_type": token.token_type,
    "access_token": token.access_token,
    "refresh_token": token.refresh_token,
    "scope": scopes[0].replace("[", "").replace("]", "").replace("'", "").split(", ") if len(scopes) == 0 else scopes,
    "expires_at": token.expires_at,
    "uses_pkce": token.uses_pkce,
    "expires_in": 0
}
json_object = json.dumps(token_dict, indent=4)
with open(".spotify_credentials.json", "w") as outfile:
    outfile.write(json_object)

# Spotify API

In [4]:
spotify.token = cred.refresh(spotify.token)

In [5]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
[track.name for track in tracks]

['Wall Of Bass',
 'When It Rains, It Pours (feat. Coco, Mr Traumatik & Songer) - Shapes Remix',
 'Bring Sally Up - Workout',
 'WKND!',
 'What Happened',
 'Wonderwall',
 'Nachts wach (Lila Wolken Bootleg)',
 'Wasted Time',
 'We Are Fighters',
 "Won't Forget",
 'Before I Wake',
 'Separate Ways (Worlds Apart)',
 "Before You Walk With Kings - Brennan Heart's Tribute Mix",
 'The World Is Yours',
 'Wait For Me',
 'We Live Forever',
 'Wildberry Lillet',
 'Weekend Party',
 'WORTH NOTHING - Fast & Furious: Drift Tape/Phonk Vol 1',
 "God's Wrath",
 'Watermelon Sugar',
 "STAR WALKIN' (League of Legends Worlds Anthem)",
 'Without Me',
 'Willst du',
 'Wake Me Up',
 'Where Are You Now',
 "I Ain't Worried",
 'All I Want for Christmas Is You',
 'Nachts wach',
 'We Made It',
 'Waiting For Love',
 'Sweater Weather',
 'Wonderful Dream (Holidays Are Coming) - Radio Version',
 'Where Did You Go? (feat. MNEK)',
 'Another Love',
 "Why'd You Only Call Me When You're High?",
 "Creepin' (with The Weeknd & 21 Sa

In [6]:
from project.util import Partition

artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists: List[tk.model.FullArtist] = list(artist for partition in artists for artist in partition)
len(artists)

78

In [7]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials
from lastfm import LastFmScraper, LastFmProxy

creds = LastFmCredentials(env["LASTFM_API_KEY"], env["LASTFM_SHARED_SECRET"])
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

lastfm = LastFmProxy(last_network, LastFmScraper())

In [8]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

In [9]:
from project.track import TaggedTrack
for track in analyzed_tracks.tracks:
    try:
        tags = TaggedTrack(lastfm, track.name, track.artist_names).tags()
        if len(tags) == 0:
            raise Exception("No tags")
    except Exception as e:
        print(e, track.id, track.name, track.artist_names)
        continue
    track.tags = tags

No tags 6IB7b4cXvYPhQsQHdmn1uE Bring Sally Up - Workout ['HIIT BPM']
No tags 5Zlb01Jcn0Ld49zazzZJSB WORTH NOTHING - Fast & Furious: Drift Tape/Phonk Vol 1 ['Oliver Tree', 'TWISTED']
No tags 0h14I7cPYi1RZG6GmVpMso Weg von mir ['CIVO']


In [10]:
print(analyzed_tracks.tracks[0].tags)

['rap', 'hardcore', 'gabber', 'dutch']


In [11]:
from time import sleep

track_ids = [track.id for track in analyzed_tracks.tracks]
track_id_partitions = Partition(track_ids)
features = [spotify.tracks_audio_features(ids) for ids in track_id_partitions]
features = [feature for partition in features for feature in partition]

for track in analyzed_tracks.tracks:
    successful = False
    analysis = None
    while not successful:
        try:
            analysis = spotify.track_audio_analysis(track_id=track.id)
            successful = True
        except httpx.HTTPError:
            print('ReadTimeout')
            successful = False
            sleep(3)

    feature = [feature for feature in features if feature.id == track.id][0]
    track.acousticness = feature.acousticness
    track.pitches = { str(segment.start):[p for p in segment.pitches] for segment in analysis.segments}
    track.loudness = feature.loudness
    track.energy = feature.energy
    track.danceability = feature.danceability
    track.mode = feature.mode
    track.instrumentalness = feature.instrumentalness
    track.key = feature.key
    track.liveness = feature.liveness
    track.tempo = feature.tempo
    track.time_signature = feature.time_signature
    track.valence = feature.valence

ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout
ReadTimeout


In [12]:
print(analyzed_tracks.tracks[0])

AnalyzedTrack(id=6MM8Rkumhy4RguXgtHCVk3, name=Wall Of Bass, tags=['rap', 'hardcore', 'gabber', 'dutch'], duration=264736, artist_genres={'rawstyle', 'frenchcore', 'uptempo hardcore'}, acousticness=0.0318, pitches={'0.0': [0.904, 0.846, 0.654, 0.561, 0.553, 0.603, 0.654, 0.719, 0.829, 0.848, 0.865, 1.0], '0.31342': [0.932, 0.861, 0.665, 0.558, 0.543, 0.604, 0.681, 0.751, 0.833, 0.848, 0.865, 1.0], '0.62694': [0.942, 0.869, 0.675, 0.583, 0.556, 0.622, 0.659, 0.739, 0.842, 0.851, 0.865, 1.0], '0.94617': [0.926, 0.859, 0.656, 0.559, 0.549, 0.596, 0.66, 0.729, 0.817, 0.826, 0.856, 1.0], '1.25991': [0.941, 0.879, 0.677, 0.571, 0.555, 0.601, 0.649, 0.72, 0.823, 0.836, 0.858, 1.0], '1.56159': [0.983, 0.921, 0.72, 0.62, 0.591, 0.627, 0.666, 0.749, 0.845, 0.848, 0.856, 1.0], '1.88667': [0.956, 0.885, 0.676, 0.581, 0.563, 0.61, 0.668, 0.736, 0.823, 0.839, 0.852, 1.0], '2.2059': [1.0, 0.894, 0.546, 0.611, 0.603, 0.524, 0.462, 0.649, 0.593, 0.574, 0.543, 0.531], '2.52639': [1.0, 0.922, 0.598, 0.438

# Firebase

In [13]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app()

In [14]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.tracks[0].upsert(firestore_client.collection("tracks"))

In [15]:
analyzed_tracks.tracks[0].id

'6MM8Rkumhy4RguXgtHCVk3'